# Groth16 Algorithm Implementation

### System Setup

In [1]:
pip install py_ecc


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install numpy=='1.26.0'


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import sys

print(f"Python Version: {sys.version}")
# For just the version number without additional info:
print(f"Version Number: {sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}")

import numpy as np

Python Version: 3.11.11 (main, Dec  3 2024, 17:20:40) [Clang 16.0.0 (clang-1600.0.26.4)]
Version Number: 3.11.11


In [4]:
np.version.version == '1.26.0'

True

### R1CS to QAP:

In [5]:
# 1, out, x, y, v1, v2, v3
L = np.array([
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, -5, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1],
])

R = np.array([
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
])

O = np.array([
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 1],
    [0, 1, 0, 0, 0, -1, 0],
])

x = 4
y = -2
v1 = x * x
v2 = v1 * v1        # x^4
v3 = -5*y * y
z = v3*v1 + v2    # -5y^2 * x^2

# witness
a = np.array([1, z, x, y, v1, v2, v3])

assert all(np.equal(np.matmul(L, a) * np.matmul(R, a), np.matmul(O, a))), "not equal"

from py_ecc.bn128 import curve_order

import galois

GF = galois.GF(curve_order, primitive_element=5, verify=False)

L = (L + curve_order) % curve_order
R = (R + curve_order) % curve_order
O = (O + curve_order) % curve_order

L_galois = GF(L)
R_galois = GF(R)
O_galois = GF(O)

x = GF(4)
y = GF(-2 + curve_order) # we are using 79 as the field size, so 79 - 2 is -2
v1 = x * x
v2 = v1 * v1         # x^4
v3 = GF(-5 + curve_order)*y * y
out = v3*v1 + v2    # -5y^2 * x^2

witness = GF(np.array([1, out, x, y, v1, v2, v3]))

LHS = np.matmul(L_galois, witness) * np.matmul(R_galois, witness)
RHS = np.matmul(O_galois, witness)
assert all(np.equal(LHS, RHS)), "not equal"

def interpolate_column(col):
    xs = GF(np.array([1,2,3,4]))
    return galois.lagrange_poly(xs, col)

# axis 0 is the columns.
# apply_along_axis is the same as doing a for loop over the columns and collecting the results in an array
U_polys = np.apply_along_axis(interpolate_column, 0, L_galois)
V_polys = np.apply_along_axis(interpolate_column, 0, R_galois)
W_polys = np.apply_along_axis(interpolate_column, 0, O_galois)
from functools import reduce

def inner_product_polynomials_with_witness(polys, witness):
    mul_ = lambda x, y: x * y
    sum_ = lambda x, y: x + y
    return reduce(sum_, map(mul_, polys, witness))

term_1 = inner_product_polynomials_with_witness(U_polys, witness)
term_2 = inner_product_polynomials_with_witness(V_polys, witness)
term_3 = inner_product_polynomials_with_witness(W_polys, witness)

# t = (x - 1)(x - 2)(x - 3)(x - 4)
t = galois.Poly([1, curve_order - 1], field = GF) * galois.Poly([1, curve_order - 2], field = GF) * galois.Poly([1, curve_order - 3], field = GF) * galois.Poly([1, curve_order - 4], field = GF)
h = (term_1 * term_2 - term_3) // t

# print("Term 1: ", term_1)
# print("Term 2: ", term_2)
# print("Term 3: ", term_3)
# print("H: ", h)
# print("T: ", t)

h = (term_1 * term_2 - term_3) // t
# print("Is it balanced? ", term_1 * term_2 == term_3 + h * t)

In [6]:
from py_ecc.bn128 import G1, G2, add, multiply, pairing, neg, eq

import random
tau = random.randint(1, curve_order - 1)
# print("Tau: ", tau)

# Number of rows is 4 so it will be degree 3
SRS1G1 = [multiply(G1, tau**3), multiply(G1, tau**2), multiply(G1, tau), G1]
SRS1G2 = [multiply(G2, tau**3), multiply(G2, tau**2), multiply(G2, tau), G2]
SRS2G1 = [multiply(G1, int(tau**2*t(tau))), multiply(G1, int(tau*t(tau))), multiply(G1, int(t(tau)))]

# print("SRS1G1: ", SRS1G1)
# print("SRS1G2: ", SRS1G2)
# print("SRS2G1: ", SRS2G1)

term_1_coeffs = [0] * (4 - term_1.degree - 1) + list(term_1.coeffs)
term_2_coeffs = [0] * (4 - term_2.degree - 1) + list(term_2.coeffs)
term_3_coeffs = [0] * (4 - term_3.degree - 1) + list(term_3.coeffs)
h_coeffs = [0] * (3 - h.degree - 1) + list(h.coeffs)

# print("term_1_coeffs:", term_1_coeffs)
# print("term_2_coeffs:", term_2_coeffs)
# print("term_3_coeffs:", term_3_coeffs)
# print("h_coeffs:", h_coeffs)

In [7]:
# Calculate L_G1 (term_1)
A = None
for i in range(4):
    G1_point = multiply(SRS1G1[i], int(term_1_coeffs[i]))
    if A is None:
        A = G1_point
    else:
        A = add(A, G1_point)

# Calculate R_G2 (term_2)
B = None
for i in range(4):
    G2_point = multiply(SRS1G2[i], int(term_2_coeffs[i]))
    if B is None:
        B = G2_point
    else:
        B = add(B, G2_point)

# Calculate O_G1 (term_3)
O_G1 = None
for i in range(4):
    G1_point = multiply(SRS1G1[i], int(term_3_coeffs[i]))
    if O_G1 is None:
        O_G1 = G1_point
    else:
        O_G1 = add(O_G1, G1_point)

# Calculate H_G1 using SRS2G1
H_G1 = None
for i in range(3):
    G1_point = multiply(SRS2G1[i], int(h_coeffs[i]))
    if H_G1 is None:
        H_G1 = G1_point
    else:
        H_G1 = add(H_G1, G1_point)

# Calculate C = O_G1 + H_G1
C = add(O_G1, H_G1)

# print("A: ", A)
# print("B: ", B)
# print("C: ", C)

eAB = pairing(B, A)
eCG2 = pairing(G2, C)

# print("eAB: ", eAB)
# print("eCG2: ", eCG2)

# Verify the relation using pairings
# e(A, B) = e(C, G2)
result = eAB == eCG2
print("Pairing verification result:", result)

Pairing verification result: True


### Complete Trusted Setup

In [8]:
# Generate random values for the trusted setup
alpha = random.randint(1, curve_order - 1)
beta = random.randint(1, curve_order - 1)

# print(f"Trusted setup parameters (normally kept secret):")
# print(f"tau = {tau}")
# print(f"alpha = {alpha}")
# print(f"beta = {beta}")

# Compute psi values
psi = []
for i in range(len(witness)):
    u_i_at_tau = int(U_polys[i](tau))
    v_i_at_tau = int(V_polys[i](tau))
    w_i_at_tau = int(W_polys[i](tau))
    
    psi_value = (w_i_at_tau + alpha * v_i_at_tau + beta * u_i_at_tau) % curve_order
    psi.append(multiply(G1, int(psi_value)))

# print("Psi value:", psi)

In [9]:
A_original = A
B_original = B

alpha_G1 = multiply(G1, alpha)
A = add(alpha_G1, A)

beta_G2 = multiply(G2, beta)
B = add(beta_G2, B)

C = None
for i in range(len(witness)):
    if witness[i] != 0:
        term = multiply(psi[i], int(witness[i]))
        if C is None:
            C = term
        else:
            C = add(C, term)

C = add(C, H_G1)

print("A: ", A)
print("B: ", B)
print("C: ", C)

A:  (3623748540628731533287469443591030686243838852342428680817550239960778790283, 19874168650910557043580459566071292864769641512892974397631644340142113836566)
B:  ((18757324239520708136581767404500635440769813539052370280912393673942320628684, 16264244504944106321790297735514507502656408693819376837572278655006716947108), (4449263999324915893942894022382500852229852392129143169976946923149662452796, 7183343093553676666670207637063161128489274195479452494428362109626801011560))
C:  (6781820513385276233619697989327723575888013024447108259516446386645021361593, 3793177191666303912885373641309590466521045707135026150348552398232240073608)


In [10]:
from py_ecc.bn128.bn128_curve import FQ12 
from py_ecc.bn128 import eq, neg, final_exponentiate

result_groth16 = eq(FQ12.one(),
   final_exponentiate(pairing(B, neg(A)) * pairing(beta_G2, alpha_G1) * pairing(G2, C)))

print(f"Groth16 verification: {result_groth16}")

Groth16 verification: True


### Add γ and δ to Trusted Setup

In [11]:
# Missing in trusted setup: gamma and delta
gamma = random.randint(1, curve_order - 1)
delta = random.randint(1, curve_order - 1)

print(f"Trusted setup parameters (normally kept secret):")
print(f"tau = {tau}")
print(f"alpha = {alpha}")
print(f"beta = {beta}")
print(f"gamma = {gamma}")
print(f"delta = {delta}")

# Public input
l = 2

psi = []
for i in range(len(witness)):
    u_i_at_tau = int(U_polys[i](tau))
    v_i_at_tau = int(V_polys[i](tau))
    w_i_at_tau = int(W_polys[i](tau))
    
    psi_value = (w_i_at_tau + alpha * v_i_at_tau + beta * u_i_at_tau) % curve_order

    denominator = None
    if i < l:
        denominator = gamma
    else:
        denominator = delta

    assert denominator != None, "Denominator cannot be `None`"
        
    inverse_denominator = pow(denominator, -1, curve_order)
    psi.append(multiply(G1, int((psi_value * inverse_denominator) % curve_order)))

print("Psi value:", psi)

inv_delta = pow(delta, -1, curve_order)
SRS2G1_delta = [multiply(G1, int(tau**2*t(tau)*inv_delta)), multiply(G1, int(tau*t(tau)*inv_delta)), multiply(G1, int(t(tau)*inv_delta))]

print("SRS1G1: ", SRS1G1)
print("SRS1G2: ", SRS1G2)
print("SRS2G1: ", SRS2G1_delta)

Trusted setup parameters (normally kept secret):
tau = 14277529767012459229562231478562599096457580399158049538453796333173532439257
alpha = 10272781246187345254272801447019002815561430849264702818045510457393049923084
beta = 10358510159581754893434049212261816132203623072703583416461504371788732263220
gamma = 3848709951053146648518175379007210740445609456197998893965841331363319949390
delta = 12435658635838886079741701901168320232773901488439002212479363614054101283433
Psi value: [None, (21808874659786183461376610409556390930255822057610130071635700110167644767832, 7380144402126394410990719927509078955658200030034342363020277193164171635327), (11612376812154477969538611170343851327711472515672828585816101515020792591920, 9120815970306982572097559080848134313449103379630603050329886753237300514133), (6484234573585343337198556851562565096851668877795699490701726260464764063350, 16544034472245882815662693254486984214553008327144434429229823209474615465318), (36032851950506665539335307902

In [12]:
A = add(alpha_G1, A_original)
B = add(beta_G2, B_original)

C = None
for i in range(l, len(witness)):  # Start from l, not 0
    if witness[i] != 0:
        term = multiply(psi[i], int(witness[i]))
        if C is None:
            C = term
        else:
            C = add(C, term)

H_G1 = None
for i in range(3):
    G1_point = multiply(SRS2G1_delta[i], int(h_coeffs[i]))
    if H_G1 is None:
        H_G1 = G1_point
    else:
        H_G1 = add(H_G1, G1_point)

C = add(C, H_G1)

print("A: ", A)
print("B: ", B)
print("C: ", C)

A:  (3623748540628731533287469443591030686243838852342428680817550239960778790283, 19874168650910557043580459566071292864769641512892974397631644340142113836566)
B:  ((18757324239520708136581767404500635440769813539052370280912393673942320628684, 16264244504944106321790297735514507502656408693819376837572278655006716947108), (4449263999324915893942894022382500852229852392129143169976946923149662452796, 7183343093553676666670207637063161128489274195479452494428362109626801011560))
C:  (11633130140171487057727548504299235523576952182511328756773729470166839847068, 18500835781368348526273986169988069345048428843081919305844350078755495267460)


### Verifier needs to compute X for public inputs
```
[X]1 = sum(ai*Ψi for i in range(1, ℓ+1))
```

### Modified verification equation
```
[A]1 ∙ [B]2 ?= [α]1 ∙ [β]2 + [X]1 ∙ [γ]2 + [C]1 ∙ [δ]2
```

In [13]:
X = multiply(G1, 0)
for i in range(l):
    if witness[i] != 0:  # Only add non-zero terms
        term = multiply(psi[i], int(witness[i]))
        X = add(X, term)

gamma_G2 = multiply(G2, gamma)
delta_G2 = multiply(G2, delta)

verification = final_exponentiate(
    pairing(B, neg(A)) *
    pairing(beta_G2, alpha_G1) *
    pairing(gamma_G2, X) * 
    pairing(delta_G2, C)
)

result_groth16 = eq(FQ12.one(), verification)

print(f"Groth16 verification: {result_groth16}")

Groth16 verification: True


### Implement Randomization

#### Missing in prover steps:
```
r = random.randint(1, curve_order - 1)
s = random.randint(1, curve_order - 1)
```

#### Modified A, B, C calculations
```
[A]1 = [α]1 + sum(ai*ui(τ)) + r*[δ]1
[B]2 = [β]2 + sum(ai*vi(τ)) + s*[δ]2
[C]1 = sum(ai*[Ψi]1) + h(τ)t(τ) + [A]1*s + [B]1*r - rs*[δ]1
```

In [14]:
r = random.randint(1, curve_order - 1)
s = random.randint(1, curve_order - 1)

print(f"Zero-Knowledge Randomization (r and s):")
print(f"r = {r}")
print(f"s = {s}")

r_delta_G1 = multiply(multiply(G1, delta), r)
A = add(r_delta_G1, A) # Add to `A` from Part 1

s_delta_G2 = multiply(multiply(G2, delta), s)
B = add(s_delta_G2, B) # Add to `B` from Part 1

B_original_G1 = None
for i in range(4):
    G1_point = multiply(SRS1G1[i], int(term_2_coeffs[i]))
    if B_original_G1 is None:
        B_original_G1 = G1_point
    else:
        B_original_G1 = add(B_original_G1, G1_point)

beta_G1 = multiply(G1, beta)
B_prime_G1 = add(beta_G1, B_original_G1)

s_delta_G1 = multiply(multiply(G1, delta), s)
B_prime_G1 = add(s_delta_G1, B_prime_G1)

C = add(C, multiply(A, s)) # Add to `C` from Part 1
C = add(C, multiply(B_prime_G1, r))

delta_G1 = multiply(G1, delta)
C = add(C, neg(multiply(delta_G1, (r * s) % curve_order)))

print("A: ", A)
print("B: ", B)
print("C: ", C)

Zero-Knowledge Randomization (r and s):
r = 14715402357651101275430805495350098371101465238898504437874753864380695232798
s = 9810369778940442665413920265710327369027796617452462786393468934085190531599
A:  (14581234463055099536391505147458469671450309058917875212389346630524632107344, 12281785149916147784692604782969566076151713816228919379886111170284362732509)
B:  ((1533039842950631868207473618356223489421777276418504132557326938181564100647, 7977771766825512297544091923285155578354014480317181378963198128206628239192), (2535990039572488667108772654057136970071804414572059535442657313609625558087, 4853582790638354540074902518298946436009283620630614158384600892730440948391))
C:  (8002318613068272157671567111578856792031212361610914470353396927372384064881, 10455566691737237992608879915144617408736070716145169051081689870159444071139)


In [15]:
verification = final_exponentiate(
    pairing(B, neg(A)) *
    pairing(beta_G2, alpha_G1) *
    pairing(gamma_G2, X) * 
    pairing(delta_G2, C)
)

result_groth16 = eq(FQ12.one(), verification)

print(f"Groth16 verification: {result_groth16}")

Groth16 verification: True
